In [1]:
import nltk
nltk.download('rslp')
nltk.download('stopwords')
import re
import codecs
import unicodedata
from unicodedata import normalize

[nltk_data] Downloading package rslp to C:\Users\Uciolli
[nltk_data]     Betelli\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Uciolli
[nltk_data]     Betelli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
class NLP_Kit(object):
	#construtor da classe
	def __init__(self, stopwordsFile = None):
		'''Caso voce queira passar um arquivo de stopwords proprio,
		utilize-o durante a construcao do objeto NLP_kit com seu caminho
		absoluto'''
		self.stemmer = nltk.stem.RSLPStemmer()
		self.sent_tokenizer = None
		self.stopwords = None
		self.stopwordsFile = stopwordsFile

	def normaliza(self, texto):
		'''Retira acentos do texto e o retorna
		'''
		try:
			return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore')
		except:
			return unicodedata.normalize('NFKD', texto.decode('UTF-8')).encode('ASCII', 'ignore')
		raise Exception('Erro no formato da string. Utilize unicode utf-8')

	def separa_frases(texto):
		'''
		Recebe um texto. Retorna lista com as frases contidas no texto que possuem mais
		de 1 caracter
		'''
		#adiciona espacos entre pontuacoes, para facilitar o trabalho do sent_tokenizer.
		texto = re.sub('([.,!?()])', r' \1 ', texto)
		texto = re.sub('\s{2,}', ' ', texto)

		if self.sent_tokenizer is None:
			self.sent_tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle')
		sentencas =  self.sent_tokenizer.tokenize(texto)
		sentencas = [sentenca for sentenca in sentencas if len(sentenca) > 1]
		return sentencas

	def tokenize(self, frase):
		'''Recebe uma frase. Retorna lista de tokens (palavras) da frase
		que sao separadas por espaco ou virgula, travessao etc
		Retorna em Unicode e minusculo
		'''
		frase = frase.lower()
		if not isinstance(frase, str):
			frase = frase.decode('utf-8')
		return re.findall(r'\w+', frase,flags = re.UNICODE)

	def stemiza(palavra):
		'''Recebe uma palavra. Retorna seu stem
		'''
		return nltk.stemmer.stem(palavra)

	def remove_stop_words(self, texto):
		'''Retorna lista de palavras sem stopwords 
		'''
		if self.stopwords is None:
			if self.stopwordsFile is not None:
				stopwords = [word for word in codecs.open(stopwordsFile, 'r', 'utf-8')]
			else:
				stopwords = nltk.corpus.stopwords.words('portuguese')

			stopwords = [str( self.normaliza(item), "utf-8" ) for item in stopwords]

			palavras = self.tokenize(texto)
			
			#print type(stopwords[1])
			palavras_importantes = [palavra for palavra in palavras if palavra.lower() not in stopwords]
		return palavras_importantes

	def palavras_mais_frequentes(self, texto):
		'''retorna lista de tuplas <palavra, frequencia>
		'''
		palavras = self.remove_stop_words(texto)

		fd = nltk.FreqDist(palavras)
		frequencias = []
		for palavra in fd:
			frequencias.append( (palavra, fd[palavra]) )
		return frequencias

	def palavras_mais_frequentes2(self, texto): #com stopwords
		'''retorna lista de tuplas <palavra, frequencia>
		'''
		palavras = texto.split()

		fd = nltk.FreqDist(palavras)
		frequencias = []
		for palavra in fd:
			frequencias.append( (palavra, fd[palavra]) )
		return frequencias

	def remove_palavra_stop_words(self, texto):#retorna frase
		'''Retorna palavras sem stopwords 
		'''
		if self.stopwords is None:
			if self.stopwordsFile is not None:
				stopwords = [word for word in codecs.open(stopwordsFile, 'r', 'utf-8')]
			else:
				stopwords = nltk.corpus.stopwords.words('portuguese')

			stopwords = [str( self.normaliza(item), "utf-8" ) for item in stopwords]
			#print(stopwords)    #mostra quais sao os stopwords retirados
			palavras = texto.split(' ')

			frase_sem_sw= ''
			
			for palavra in palavras:
				if palavra not in stopwords:
					frase_sem_sw += palavra+' '
		return frase_sem_sw

	def remove_pontuacao(self, texto):#retorna frase
		string_normalizada = re.sub(':|;|\[|\]|\(|\)|,|\.|!|\?|\{|\}|\*|\"|-', ' ', texto.replace('\n',' '))	#retira pontuacao
		string_normalizada = string_normalizada.replace('//',' ')
		string_normalizada = string_normalizada.replace('\'','')
		return string_normalizada

In [3]:
texto = "Teste De verificação :) de um Tweet teste!"
frases = NLP_Kit()
#string_normalizada = frases.remove_pontuacao(texto)
#print(string_normalizada)
#string_normalizada = frases.normaliza(string_normalizada)   #retira acentos
#string_normalizada = str( string_normalizada, "utf-8" )
#print(string_normalizada)
#string_normalizada = frases.remove_palavra_stop_words(string_normalizada.lower())
#print(string_normalizada)
#string_normalizada = re.sub(' +', ' ', string_normalizada) #remove espacos em branco multiplos
#print(string_normalizada)
#string_normalizada = ((unicodedata.normalize('NFKD', string_normalizada)).encode('ASCII', 'ignore').lower())
#print(string_normalizada)
#string_normalizada = frases.palavras_mais_frequentes(string_normalizada)
#print(string_normalizada)

In [4]:
import csv
with open('tweets_ofensivos_sem_repeticao.csv', encoding="utf8") as f:
    reader = csv.reader(f)
    lista = list(reader)

In [5]:
lista_processada = []
for row in lista:
    texto_processando = frases.remove_pontuacao(row[0])
    texto_processando = frases.normaliza(texto_processando)
    texto_processando = str( texto_processando, "utf-8" )
    texto_processando = frases.remove_palavra_stop_words(texto_processando.lower())
    texto_processando = re.sub(' +', ' ', texto_processando)
    lista_processada.append(texto_processando)
    print(texto_processando)

text 
rt @mariama59879065 tvi ficava melhor prolongado rs durante 1 mes porque publico agarrado concorr... 
= mang trong minh dong mau lac hong phai cu goi la quen thuoc voi kieu nay https t co/ostdcs3pvw 
tomamo cu 
n entendo oq passa cabeca dessas pessoas namoral pq alem d levar hate pra leva tb pro proprio fav vao tomar cu chatice 
vai tomar cu professor filho puta 
rt @thulinh_2k2 nua ang ngu ma bi thang trai ut cu vao lon khi k hay https t co/vjiihbwbab 
caralho aleluia vai tomar cu everton 
rt @nhinhi7777 nhoc trai thang hoc spkt tim zalo qanh ay thay ga cho gay bu thu ngai k dam ga 2 3 ngay cuoi cung cung qa con cu... 
mucalol rico aq dando dinheiro pra filho puta burro chat vai tomar cu mucalol 
rt @wtfjunkook comprei armario pra guardar coisas album entra pouco vai tomar cu https t co/41ikwbhi9z 
tiraram dia pra irritar vai tomar cu 
aja cu pra tomar parcero 
 vai tomar cu faz motivos pra estar fodida ideias hj dia https t co/uj3mfkh1ke 
@comidasestadios @santosfc @brunoovazz 

fico doida maluca doidinha https t co/itfzkvgj9c 
@favortgiri @natielevadia negocio su1cidio mt doido 
@padree7bot doido acho q curte tobo gigante batendo 
gente quw role doido 
@centralreality comeco bbb amava scooby ver antijogo deu pouco raiva amar scooby ta doido queria ser assim 
mano ontem tava cheio ta doido 
@thiagokrause2 amigo doido florida ta mirando disney revogou decretos q davam poderes empresa sobre areas onde funcionam pq posicionou pressionada contra lei palanque vai custar caro tax payers parecem preocupados menos n 
pqp acredito vi gente acusando plagio causa data lancamento kkkkkkkkkk juro cara rindo desse povo doido mds 
ainda barracao doido pra chegar casa 
@cabralxsp doido 
@viviane61560441 @lmpicon obvio pra atacar pessoa q fez absolutamente nada contra fav pra mim nenhum sentido atacou pessoas ligadas mae filho vai pagar pato vc colocando carater pa cheque justificativa defesa triste 
todo dia lembro paguei 120 reais nessa blusa feia flanela pq mae disse bonita

simplesmente bebo porem sensacao acordar tarde ressaca escrota fudida 
@corinthiana ai fica gente idiota esperando alguem dar cara pra falar nome grupo pode causa campanha escrota dessas 
@analubex gente escrota perdem nocao pq anonimo pergunta q respondo maioria respondo deboche tbm pq gnt q merece saber oq passa vida gnt sinto mto perguntas q fizeram pra vc 
saudadezinhaaa escrota 
@xrayvelten @babadofamososrj vtnc fake escrota crlho 
@therealganjagrl fuzil kkkkk escrota demais 
querendo n querendo q escrota cm causa porra inseguranca 
odio gigante macho escroto nojento 
rt @haroldonalata confesso fiquei constrangida coragem assistir video homem escroto pai 
@03302000l caras escroto pior ainda anda pai crianca kkkkkkk 
@beaaacomentah ahhh va... pessoa sofro depressao sei doenca justifica falta carater nenhuma hipotese parem passar pano pra macho escroto amor 
snrz ta escroto bglh pcr https t co/gvl43a218p 
@leandro_dtm @cronicasccp bando escroto intolerantes cagam boca pregam vivemo 

@suarezstela @eltiempo existe una seccion en los medios informativos llama internacional incluso justicia mujer tenga un poco empatia vieja estupida 
rt @jesus_pinero @darvinsonrojas esto parece poco etico sometes estos chamos al escarnio publico porque evidentemente tienen argu... 
rt @liligcancun los medios y los poderes detras los conservadores buscaron demeritar toda accion enfocada sanar y remendar el tejid... 
acho vai provar desse hate trian sabe jogar cai jogo apelativa faca diss alto nivel fas querem parece quer ver nicki lama quinto album fracasso geral esperando nm5 https t co/kg7qdmm3a4 
pessoa saiu cmg vez ta chamando namorado falando amo cai q emocao toda hein 
@saudeprefsp teste pra maluco pessoa quiser saber disso ai cai 
@sigagazetabr cai ex mamateiro 
@abrahamweint quer ser politico aguenta tranco cai 
@4mydopix entra fia cai tira sujeira cara 
@feh__amaro brasil produz pior cafe todos tomarmos quantidade rato fragmentos insetos exportar qualidade chega ser monstruosa

nojento https t co/vfa4c7yyxx https t co/4djtrw8gy0 
@o_fisicoteorico @pachexeca nojento mostra desejo disso ta mim nojento 
goool porraaaaaaaa vtnc everton time nojento 
sobre tirar foto nojento mendigo falou ata #bbb22 https t co/xokaft8igq 
enfia convite rabo nojento crlho https t co/kpoxpnwfip 
rt @somosaopaulinos rogerio ceni falou sobre expulsao quarto arbitro fez put sacanagem sabe vai acontecer ... 
trabalhar durante 3 dias semana vai ser doido 
impossivel ser ano 2023 certo to bem curioso pra saber itinerario sedenta pra trabalhar otr ii + gemeos + blue crescida vai querer ser flexivel https t co/4iqopxflnm 
q odio dessa loira oxigenada caralho vai trabalhar p comprar coisa pagar outrossssss 
@drpaulofaria22 desespero vai trabalhar produzir algo grandes marcas interessam dar patrocinio abandone profissao 
@jornaleira_ @lvbcxx gente vai ter trabalhar 
bebek vai voltar trabalhar to feliz triste porque consegui desapegar ozan ainda precisa trabalhar pra sustentar atlas ada leite 

...

rt @mumiaabujamal happy birthday mumia #freemumianow #freemumia #40yearsinprison #mumiaisinnocent #freehimnow 
rt @kamaufranklin like all empires the harsh &amp violent forms of control that have been used on the wretched of the earth have migrate... 
rt @g_ricordeau mumia abu jamal fete son 68e anniversaire en prison apres plus 40 ans derriere les barreaux #freemumianow #freemumia... 
@marinasilva ai mumia ta 
rt @vezapurpura los andaluces pagaron villen psoe 2 500 en hoteles 21 000 en restaurantes 33 000 en prostibulos&amp #8230 https ... 
rt @artofne entiendo puede estar tan generalizado argumento en serio decidme 3 hombres los les haya arruinado la v... 
rt @tresmasciento el socialismo arruinado continentes devastado generaciones destruido economia y bienestar social todos los paise... 
es normal mi finde haya arruinado ir un futbol ayer literal re chinchudo ando 
oh acaban meter en el grupo whatsapp la carrera dia arruinado 
rt @centraldepp perdi tudo importa resultado deste julga

rt @tirodemeta3 facil pedir demissao rogerio ceni vai trazer atuais condicoes clube cuca vem aguirre talv... 
@paparazzorn time vindo duas otimas atuacoes ser falando asneira dessa pqp pra mim hoje ver rmp falar irritante presuncoso aposentar 
aguento arrocha brega funk torando ultimo volume casa vizinho agora trap trap coisa desocupado vagabundo mau carater trambiqueiro gente ma indole vou ligar pra policia 
@senadorhumberto ah imbecil desocupado 
bando desocupado pqp hein 
@xaviervidalb @lassoguillermo usted anda desocupado recoja firmas el ciudadano trabaja y anda en pleitos politicos 
rt @v111ni todos livros isaac apareceu lendo heartstopper lista pq desocupado https t co/dyur5ypbm9 
aaaaaaah vai toma cu povo cara kkkkkkkkkkkkkkkkkkk bando desocupado amigos familia https t co/ijcpqyxwmt 
@douglas_sena7 @bia_qrzx acha deus desocupado igual pra ficar preocupando euhem vai tratar 
@favslouisst91 amg vou olhar novo la li maioria mto desocupada kkkkkkkkkk 
 gosto vc foda filha gosta pq 

rt @fodasemeumano acho 100% pessoas relacionei 99 999% alguma forma sacana comigo interessa... 
album vivo sacana tao bom 
wooyoung deixa folego mandando beijo longe gosta fazer carinho coxa durante beijo fica observando pessoa entao puxa beijar beija tao bem treme ainda arfa durante vezes olhar sorrisinho sacana https t co/wz9navaafr 
@jnkrocksz bisha aespa surra fav meia boca ai testuda fanha muda tornozelo pacoca bp surra coisa nenhuma jisso ai q eh fa sabe cantar direito rose ai sabe gemer palco resumindo fav eh puro fracasso aespa pisa nelas 
@itsfuyuuka ah testuda 
rt @vitoriabenelli_ @brunaluana__ feia testuda 
@brunaluana__ feia testuda 
pisou testuda toda natural https t co/zztlrmjbcg 
gamora nova mo testuda 
@bassettversion vergonha oq mona fan testuda fan dessa bomba serie perfil horroroso menos mlr va lavar louca 
@laraponttes @yasabreuu @ohashi_jun_ @janejjay vai foder testuda 
@lisaboazuda @parkgifs @tracklist testuda feia icon querendo falar alguma coisa https t co/vznqx

@jspilarii vosses ofendero mossa cambada macho iscroto https t co/xqyq676c4p 
@luisferdobairro @virgini56614922 @sandraf62983385 @marciabagnara vamos eliminar gustavo pq macho iscroto insuportavel odeia artur morre inveja eli erros nunca votou artur 
@lafemmedemaki mondronga 
@anascontry mano esperando vc falar negocio mondronga 
caralho serviu demais amei look demais porem n atrevo colocar pq ia ficar mondronga https t co/wm105massi 
nada mondronga rafaela 
mondronga velha 
tomar cu dessa mondronga https t co/eqmwvzzid1 
@marmittasquad @onikelrey mondronga porc4 
cara dessa mondronga bixa amor deus pra falar assim rihanna https t co/xpcu6o0hca 
@marmittasquad n canso surrar mondronga https t co/lxhuln4zpn 
qria roubar marca deus q raiva dessa cachorra mondronga 
viciada chamar outros mondrongo 
@voirvampire moco mondrongo 
@o_antagonista claro pro mondrongo 
mondrongo 
costume usar wpp web mobile perco mao tamanho mensagem conta diferenca telas cada mondrongo mensagem conversa comigo 

@marcofeliciano salva eleitor pastor demo boca furico igual asno psicopata presidente vao tomar https t co/jtavmqpxny 
rt @ericlinsg magoa megalomania realmente conseguem ser escondidas tempo abraham weintraub apelar vitimism... 
@f4b10rms patetico patati infantil patata 
rt @delaespriellae nada le afecta todo le resbala puede mentir soltar incongruencias protagonizar un patetico show una firma en ma... 
@sf_moro paraaaaaaa ta feio vc amigo pra dizer quanto patetico 
@abrahamendieta ah espana nada coges @andreachaveztre crees mexicano... patetico 
@anacunetas55 patetico 
rt @fatosnacionais patetico gente quer dinheiro https t co/mehfdpsvuc 
rt @jok_ase milei tiene nada tan asi es editar imagen tiene robar una foto la plaza llenamos bancar... 
@x_snowice ya voy susurro al llegar su lado enfrentandose al wingull con su zubat el combate aereo duraria un poco ya aquellos pokemon es tuvieran ataques muy fuertes asi desde fuera podia ver algo patetico tengo mejorar un poco pero oye ahi + 
rt

In [6]:
dict_lista = {'text': lista_processada}

@jonastuita cada dia coisa 6 notaram 


In [7]:
lista_processada

['text ',
 'rt @mariama59879065 tvi ficava melhor prolongado rs durante 1 mes porque publico agarrado concorr... ',
 '= mang trong minh dong mau lac hong phai cu goi la quen thuoc voi kieu nay https t co/ostdcs3pvw ',
 'tomamo cu ',
 'n entendo oq passa cabeca dessas pessoas namoral pq alem d levar hate pra leva tb pro proprio fav vao tomar cu chatice ',
 'vai tomar cu professor filho puta ',
 'rt @thulinh_2k2 nua ang ngu ma bi thang trai ut cu vao lon khi k hay https t co/vjiihbwbab ',
 'caralho aleluia vai tomar cu everton ',
 'rt @nhinhi7777 nhoc trai thang hoc spkt tim zalo qanh ay thay ga cho gay bu thu ngai k dam ga 2 3 ngay cuoi cung cung qa con cu... ',
 'mucalol rico aq dando dinheiro pra filho puta burro chat vai tomar cu mucalol ',
 'rt @wtfjunkook comprei armario pra guardar coisas album entra pouco vai tomar cu https t co/41ikwbhi9z ',
 'tiraram dia pra irritar vai tomar cu ',
 'aja cu pra tomar parcero ',
 ' vai tomar cu faz motivos pra estar fodida ideias hj dia https t 

In [8]:
import pandas as pd
df = pd.DataFrame(dict_lista)

In [9]:
df.head()
df.count()

text    6609
dtype: int64

In [10]:
df.to_csv('tweets_ofensivos_finais_com_ofensa.csv', index=False)

In [ ]:
#df.to_csv('tweets_neutros_finais_com_coisa.csv', index=False)